# DIMA Use Case: Dictator Game

## Introduction

Given the DIMA model integration figure, we implemented a decision-making module based on the Dictator Game, that uses the active identity and salience value received from the DIMA module to determine the agent's actions (the amount of money a dictator agent gives to a receiver agent). With this scenario, we are going to run several settings.

## Preparing the Scenario

We start by running the DIMA model jupyter notebook.

In [ ]:
%run ./DynamicIdentityModelForAgents.ipynb

Then, we specifify the Dictator Game agent and scenario, storing the quantity of money collected by the agent. Given the interaction between two agents, when the dictator agent's active identity is the personal identity, this agent gives to the receiver a baseline offer (usually 20% of the initial amount). When the active identity is a social identity, in an in-group interaction the receiver agent will receive more money compared to an out-group interaction, depending this on the salience of the active identity. This offer can be formalized like this:

$offer_{t} = f(baselineOffer, inGroupBias, Salience{(SSI_{dictator})_{t}}) = baselineOffer \times (1 + inGroupBias \times Salience{(SSI_{dictator})_{t}}),$

$inGroupBias = \begin{cases} 1\text{,} & \text{if ${receiver} \in SSI_{dictator}$} \\-1\text{,} & \text{otherwise}
\end{cases}$

In [ ]:
class DictatorGameAgent:
    def __init__(self, adapting):
        self.moneyReceived = 0
        self.moneyGivenIngroupMember = 0
        self.moneyGivenOutgroupMember = 0
        self.inGroupInteractions = 0
        self.outGroupInteractions = 0
        
        self.adapting = adapting

def addScenarioAgents(agents, flagAdapt = False):
    for i, a in enumerate(agents):
        adapting = True
        if flagAdapt:
            if (i < len(agents)/2):
                adapting = False
        a.scenario = DictatorGameAgent(adapting)
        
class DictatorGame:
    def __init__(self, baselineOffer, potSize):
        self.baselineOffer = baselineOffer
        self.potSize = potSize
        
        self.groupsAverageOffersToIngroup = []
        self.groupsAverageOffersToIngroupFinalStep = []
        
        self.groupsAverageOffersToOutgroup = []
        self.groupsAverageOffersToOutgroupFinalStep = []
        
        self.groupsAverageWealthSteps = []
        self.groupsAverageWealthFinalStep = []
                
        self.adaptingGroupsAverageWealthSteps = []
        self.adaptingGroupsAverageWealthFinalStep = []
        
    def offer(self, agentOffering, agentReceiving, salience, inGroupBool):
        if salience is None or agentOffering.scenario.adapting == False:
            offer = self.baselineOffer
        else: #social identity
            if inGroupBool: #in-group favoritism
                offer = self.baselineOffer * (1 + salience)
                agentOffering.scenario.moneyGivenIngroupMember += offer
                agentOffering.scenario.inGroupInteractions += 1
            else: #out-group discrimination
                offer = self.baselineOffer * (1 - salience)
                agentOffering.scenario.moneyGivenOutgroupMember += offer
                agentOffering.scenario.outGroupInteractions += 1
        
        agentOffering.scenario.moneyReceived += self.potSize - offer
        agentReceiving.scenario.moneyReceived += offer

        #offer --> payoff --> impact - outcome evaluation based on payoff - not applied 
        #return -0.2*(offer/(self.baselineOffer*2)) + 0.1
    
    #For plots
    def updateAvgWealth(self, groupIdx, agents, s, r):
        sumMoney = 0
        lenAgents = len(agents)
        for a in agents:
            sumMoney += a.scenario.moneyReceived
        avg = sumMoney/lenAgents
        if len(self.groupsAverageWealthSteps) <= groupIdx:            
            self.groupsAverageWealthSteps.append([0] * numSteps)
        self.groupsAverageWealthSteps[groupIdx][s] += avg
        if s == (numSteps-1):
            if len(self.groupsAverageWealthFinalStep) <= groupIdx:
                self.groupsAverageWealthFinalStep.append([0] * numRuns)
            self.groupsAverageWealthFinalStep[groupIdx][r] = avg

    def updateAvgWealthAdapting(self, s, r):
        lenAdapting = 0
        sumMoneyAdapting = 0
        lenNotAdapting = 0
        sumMoneyNotAdapting = 0
        for a in context.agentsPresent:
            if a.scenario.adapting:
                sumMoneyAdapting += a.scenario.moneyReceived
                lenAdapting += 1
            else: 
                sumMoneyNotAdapting += a.scenario.moneyReceived
                lenNotAdapting += 1
        avgAdapting = sumMoneyAdapting/lenAdapting
        avgNotAdapting = sumMoneyNotAdapting/lenNotAdapting
        if len(self.adaptingGroupsAverageWealthSteps) == 0:
            self.adaptingGroupsAverageWealthSteps.append([0]*numSteps)
            self.adaptingGroupsAverageWealthSteps.append([0]*numSteps)
        self.adaptingGroupsAverageWealthSteps[0][s] += avgAdapting
        self.adaptingGroupsAverageWealthSteps[1][s] += avgNotAdapting
        if s == (numSteps-1):
            if len(self.adaptingGroupsAverageWealthFinalStep) == 0:
                self.adaptingGroupsAverageWealthFinalStep.append([0] * numRuns)
                self.adaptingGroupsAverageWealthFinalStep.append([0] * numRuns)
            self.adaptingGroupsAverageWealthFinalStep[0][r] = avgAdapting
            self.adaptingGroupsAverageWealthFinalStep[1][r] = avgNotAdapting
                
    def updateAvgOffers(self, groupIdx, agents, s, r):
        ingroupOfferTotal = 0
        outgroupOfferTotal = 0
        lenAgents = len(agents)
        for a in agents:
            if(a.scenario.inGroupInteractions != 0):
                ingroupOfferTotal += a.scenario.moneyGivenIngroupMember/a.scenario.inGroupInteractions
            if(a.scenario.outGroupInteractions != 0):
                outgroupOfferTotal += a.scenario.moneyGivenOutgroupMember/a.scenario.outGroupInteractions
        avg = ingroupOfferTotal/lenAgents
        if len(self.groupsAverageOffersToIngroup) <= groupIdx:            
            self.groupsAverageOffersToIngroup.append([0] * numSteps)
        self.groupsAverageOffersToIngroup[groupIdx][s] += avg
        if s == (numSteps-1):
            if len(self.groupsAverageOffersToIngroupFinalStep) <= groupIdx:
                self.groupsAverageOffersToIngroupFinalStep.append([0] * numRuns)
            self.groupsAverageOffersToIngroupFinalStep[groupIdx][r] = avg
        
        avg = outgroupOfferTotal/lenAgents
        if len(self.groupsAverageOffersToOutgroup) <= groupIdx:            
            self.groupsAverageOffersToOutgroup.append([0] * numSteps)
        self.groupsAverageOffersToOutgroup[groupIdx][s] += avg
        if s == (numSteps-1):
            if len(self.groupsAverageOffersToOutgroupFinalStep) <= groupIdx:
                self.groupsAverageOffersToOutgroupFinalStep.append([0] * numRuns)
            self.groupsAverageOffersToOutgroupFinalStep[groupIdx][r] = avg
    

The interaction between two agents is random (well-mixed population). 

In [ ]:
def playScenario(giver, idxGiver):
    inGroup = False
    idxReceiver = np.random.choice([i for i in range(0,len(context.agentsPresent)) if i != idxGiver])
    receiver = context.agentsPresent[idxReceiver]
    #check if in group or out group
    if clustersCt[1][idxReceiver] == clustersCt[1][idxGiver]:
        inGroup = True
    else:
        inGroup = False
    #check if salient identity of giver is empty
    if giver.salientIdentity is None:
        salience = None
    else:
        salience = giver.salientIdentity.salience
        
    return scenario.offer(giver, receiver, salience, inGroup)

## Preparing the DIMA Input

We did not implement a perception module. In this scenario, the agent's characteristics were randomized at each run of the simulation. The context can be empty or have a specific number of relevant characteristics.

In [ ]:
def prepareAgents(numA, pGroup = 1, init = False, d1 = 0.3, d2 = 0.7):
    if init:
        agents = []
    else:
        agents = context.agentsPresent
    limit = numA*pGroup
    for i in range(0, numA):
        charA = {}
        for j in range(0,2):
            if i < limit:
                charA["Characteristic " + str(j+1)] = np.around(np.random.uniform(0,d1),3)
            else:
                charA["Characteristic " + str(j+1)] = np.around(np.random.uniform(d2,1),3)
        if init:
            agents.append(Agent(str(i), charA, []))
        else:
            agents[i].personalCharacteristics = charA
    return agents
    
def contextTheme(numC):
    theme = {}
    for t in range(0, numC):
        theme["Characteristic " + str(t+1)] = 1/numC
    return theme

def prepareContext(numC):
    theme = contextTheme(numC)
    return SocialContext(agents, theme)



## Plots

In [ ]:
def auxPlotScenario(s, r, flagAdapt = False):
    for sg in range(0, clustersCt[0]):
        agentsGroup = []
        for a, c in enumerate(clustersCt[1]):
            if c == sg:
                agentsGroup.append(context.agentsPresent[a])
        scenario.updateAvgWealth(sg, agentsGroup, s, r)
        scenario.updateAvgOffers(sg, agentsGroup, s, r)
    if flagAdapt:
        scenario.updateAvgWealthAdapting(s, r)

def auxClearScenario():
    for a in context.agentsPresent:
        a.scenario.moneyReceived = 0
        a.scenario.moneyGivenIngroupMember = 0
        a.scenario.moneyGivenOutgroupMember = 0
        a.scenario.inGroupInteractions = 0
        a.scenario.outGroupInteractions = 0

Functions to show the wealth plots (over time and box plot for the final step), as well as other data.

In [ ]:
def showAverageWealthGroupsPlots():
    lenColors = len(colors)
    lenLine = len(plotLine)
    fig = plt.figure(figsize = (7,5))
    ax = fig.add_subplot(1, 1, 1)
    ax.yaxis.set_major_locator(plt.MultipleLocator(2000))
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    plotStyle(ax)
    w = []
    for i, sgWealth in enumerate(scenario.groupsAverageWealthSteps):
        colorPlot = colors[i%lenColors]
        linePlot = plotLine[i%lenLine]
        wealthSims = [x / numRuns for x in sgWealth]
        plt.plot(wealthSims, color=colorPlot, linewidth=2.5, marker=linePlot, markevery=3, label='Avg Accumulated Wealth of Agent in Cluster ' + str(i+1))
        w.append(wealthSims)
        wealthMeanStdDev(i, scenario.groupsAverageWealthFinalStep[i])
    if len(w) == 2:
        plt.fill_between(np.arange(numSteps), w[0], w[1], color="grey", alpha=0.3)
    plt.title('Average Accumulated Cluster Wealth:\nEach agent in the environment uses their active identity\nto play the Dictator Game and gather wealth')
    plt.legend(loc='upper left', fancybox=True)
    plt.xlim(0, numSteps-1)
    plt.ylim(0, 12000)
    plt.xlabel("Number of Steps")
    plt.ylabel("Money (€)")
    plt.show()
    
def wealthMeanStdDev(sg, data):
    print("Cluster " + str(sg+1))
    print("Step = 100")
    print("Wealth")
    print("Mean: " + str(meanF(data)))
    print("Standard Deviation: " + str(stdev(data)))
    
import seaborn as sns
import pandas as pd
def showAverageWealthBoxPlotClusters():
    d = {}
    c = {}
    lenColors = len(colors)
    for i, w in enumerate(scenario.groupsAverageWealthFinalStep):
        d["Cluster " + str(i+1)] = w
        c["Cluster " + str(i+1)] = colors[i%lenColors]
    df = pd.DataFrame(data=d)
    fig = plt.figure(figsize = (7,5))
    g = sns.boxplot(data=df, palette = c)
    g.set_title('Box Plot of Average Accumulated Cluster Wealth: Step = 100')
    g.set_ylabel("Money (€)")
    plotStyle(g)
    plt.show()

Functions to show plots of average in-group and out-group offers.

In [ ]:
def showAverageOffersPlots(inGroup = True):
    lenColors = len(colors)
    lenLine = len(plotLine)
    fig = plt.figure(figsize = (7,5))
    ax = fig.add_subplot(1, 1, 1)
    ax.yaxis.set_major_locator(plt.MultipleLocator(5))
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    plotStyle(ax)
    of = []
    if (inGroup):
        gAoffers = scenario.groupsAverageOffersToIngroup
    else:
        gAoffers = scenario.groupsAverageOffersToOutgroup
    for i, sgOffers in enumerate(gAoffers):
        colorPlot = colors[i%lenColors]
        linePlot = plotLine[i%lenLine]
        offersSims = [x / numRuns for x in sgOffers]
        if inGroup:
            plt.plot(offersSims, color=colorPlot, linewidth=2.5, marker=linePlot, markevery=3, label='Avg Offer of Agent in Cluster ' + str(i+1) +' to In-Group Member')
            wealthMeanStdDev(i, scenario.groupsAverageOffersToIngroupFinalStep[i])
        else:
            plt.plot(offersSims, color=colorPlot, linewidth=2.5, marker=linePlot, markevery=3, label='Avg Offer of Agent in Cluster ' + str(i+1) +' to Out-Group Member')
            wealthMeanStdDev(i, scenario.groupsAverageOffersToOutgroupFinalStep[i])
       
        of.append(offersSims)

    if len(of) == 2:
        plt.fill_between(np.arange(numSteps), of[0], of[1], color="grey", alpha=0.3)
    if inGroup:
        plt.title('Average Offer to In-Group Members:\nEach dictator agent offers a specific amount of money\nto their in-group members according to their active identity')
    else:
        plt.title('Average Offer to Out-Group Members:\nEach dictator agent offers a specific amount of money\nto their out-group members according to their active identity')
    plt.legend(loc='upper left', fancybox=True)
    plt.xlim(0, numSteps-1)
    plt.ylim(0, 50)
    plt.xlabel("Number of Steps")
    plt.ylabel("Money (€)")
    plt.show()
    

## Settings

### Baseline: No Context is Active

In [ ]:
def cycleScenario1(): 
    #Run simulation
    global clustersCt
    clustersCt = clustering()
    printClustering()
    for r in range(0, numRuns):
        for s in range(0, numSteps):       
            #For each agent
            for idxAgent, a in enumerate(context.agentsPresent):
                normativeFit(a, idxAgent)
                comparativeFit(a)
                salienceSocialGroup(a, idxAgent)
                salientActiveIdentity(a, idxAgent)
                listGraphics(a, s, r)
                playScenario(a, idxAgent) #added this function
                updateAccessibility(a) #update accessibility, no outcome evaluation was performed
            auxPlotScenario(s, r) #added this function
        clearSim()
        auxClearScenario() #added this function

In [ ]:
#Restarting the simulation
numRuns = 50
agents = prepareAgents(50, 1.0, True)
context = prepareContext(0)
addScenarioAgents(agents)
scenario = DictatorGame(20, 100)
cycleScenario1()
#Plots
print("Wealth Plots")
showAverageWealthGroupsPlots()


### Two Groups: Equal Size and Homogeneity + Context is Static

In [ ]:
def cycleScenario2(sizeP = 0.5, d1 = 0.3, d2 = 0.7):  
    #Run simulation
    for r in range(0, numRuns):
        agents = prepareAgents(50, sizeP, False, d1, d2)
        global clustersCt
        clustersCt = clustering()
        printClustering()
        for s in range(0, numSteps):      
            #For each agent
            for idxAgent, a in enumerate(context.agentsPresent):
                normativeFit(a, idxAgent)
                comparativeFit(a)
                salienceSocialGroup(a, idxAgent)
                salientActiveIdentity(a, idxAgent)
                listGraphics(a, s, r)
                playScenario(a, idxAgent) #added this function
                updateAccessibility(a) #update accessibility, no outcome evaluation was performed
            auxPlotScenario(s, r) #added this function
        clearSim()
        auxClearScenario() #added this function

In [ ]:
#Restarting the simulation
agents = prepareAgents(50, 0.5, True)
context = prepareContext(2)
addScenarioAgents(agents)
scenario = DictatorGame(20, 100)
cycleScenario2()
#Plots
print("Wealth Plots")
showAverageWealthGroupsPlots()
showAverageOffersPlots(True)
showAverageOffersPlots(False)
showAverageWealthBoxPlotClusters()
print("Salience and Accessibility Plots")
showSalienceAccessibilityPlots()

### Two Groups: Different Size (70/30) + Context is Static

In [ ]:
#Restarting the simulation
agents = prepareAgents(50, 0.7, True)
context = prepareContext(2)
addScenarioAgents(agents)
scenario = DictatorGame(20, 100)
cycleScenario2(0.7)
#Plots
print("Wealth Plots")
showAverageWealthGroupsPlots()
showAverageOffersPlots(True)
showAverageOffersPlots(False)
showAverageWealthBoxPlotClusters()
print("Salience and Accessibility Plots")
showSalienceAccessibilityPlots()

### Two Groups: Different Size (90/10) + Context is Static

In [ ]:
#Restarting the simulation
agents = prepareAgents(50, 0.9, True)
context = prepareContext(2)
addScenarioAgents(agents)
scenario = DictatorGame(20, 100)
cycleScenario2(0.9)
#Plots
print("Wealth Plots")
showAverageWealthGroupsPlots()
showAverageOffersPlots(True)
showAverageOffersPlots(False)
showAverageWealthBoxPlotClusters()
print("Salience and Accessibility Plots")
showSalienceAccessibilityPlots()

### Two Groups: Different Homogeneity + Context is Static

In [ ]:
#Restarting the simulation
agents = prepareAgents(50, 0.5, True, 0.1, 0.5)
context = prepareContext(2)
addScenarioAgents(agents)
scenario = DictatorGame(20, 100)
cycleScenario2(0.5, 0.1, 0.5)
#Plots
print("Wealth Plots")
showAverageWealthGroupsPlots()
showAverageOffersPlots(True)
showAverageOffersPlots(False)
showAverageWealthBoxPlotClusters()
print("Salience and Accessibility Plots")
showSalienceAccessibilityPlots()

### Two Equal Groups + Context Varies + Not all agents adapt

In [ ]:
def showAverageWealthAdaptingAgentsPlots():
    lenColors = len(colors)
    lenLine = len(plotLine)
    fig = plt.figure(figsize = (7,5))
    ax = fig.add_subplot(1, 1, 1)
    ax.yaxis.set_major_locator(plt.MultipleLocator(2000))
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    plotStyle(ax)
    w = []
    for i, sgWealth in enumerate(scenario.adaptingGroupsAverageWealthSteps):
        colorPlot = colors[i%lenColors]
        linePlot = plotLine[i%lenLine]
        wealthSims = [x / numRuns for x in sgWealth]
        if (i==0):
            plt.plot(wealthSims, color=colorPlot, linewidth=2.5, marker=linePlot, markevery=3, label='Avg Accumulated Wealth of Adaptive Agent')
            wealthMeanStdDevAdaptive("Adaptive Agent", scenario.adaptingGroupsAverageWealthFinalStep[i])
        else:
            plt.plot(wealthSims, color=colorPlot, linewidth=2.5, marker=linePlot, markevery=3, label='Avg Accumulated Wealth of Non-Adaptive Agent')
            wealthMeanStdDevAdaptive("Non-Adaptive Agent", scenario.adaptingGroupsAverageWealthFinalStep[i])
        w.append(wealthSims)
    if len(w) == 2:
        plt.fill_between(np.arange(numSteps), w[0], w[1], color="grey", alpha=0.3)
    plt.title('Average Accumulated Wealth of Adaptive and Non-Adaptive Agents:\nAdaptive agents may use their personal or social identity\nto play the Dictator Game and gather wealth\nNon-Adaptive agents always use their personal identity')
    plt.legend(loc='upper left', fancybox=True)
    plt.xlim(0, numSteps-1)
    plt.ylim(0, 12000)
    plt.xlabel("Number of Steps")
    plt.ylabel("Money (€)")
    plt.show()
    
def wealthMeanStdDevAdaptive(adaptive, data):
    print(adaptive)
    print("Step = 100")
    print("Wealth")
    print("Mean: " + str(meanF(data)))
    print("Standard Deviation: " + str(stdev(data)))
    
import seaborn as sns
import pandas as pd
def showAverageWealthBoxPlotAdaptingAgents():
    d = {}
    c = {}
    d["Adaptive Agents"] = scenario.adaptingGroupsAverageWealthFinalStep[0]
    c["Adaptive Agents"] = colors[0]
    d["Non-Adaptive Agents"] = scenario.adaptingGroupsAverageWealthFinalStep[1]
    c["Non-Adaptive Agents"] = colors[1]
    df = pd.DataFrame(data=d)
    fig = plt.figure(figsize = (7,5))
    g = sns.boxplot(data=df, palette = c)
    g.set_title('Box Plot of Average Accumulated Wealth\nof Adaptive and Non-Adaptive Agents: Step=100')
    g.set_ylabel("Money (€)")
    plotStyle(g)
    plt.show()
    
def cycleScenario3():
    #Run simulation
    for r in range(0, numRuns):
        agents = prepareAgents(50, 0.5)
        for s in range(0, numSteps):
            if (s%2) == 0:
                context.theme = contextTheme(2)
            else:
                context.theme = contextTheme(0)
            global clustersCt
            clustersCt = clustering()
            printClustering()
            for a in context.agentsPresent:
                for sg in a.normativeGroups:
                    if sg.type == 1:                
                        sg.type = 0
            #For each agent
            for idxAgent, a in enumerate(context.agentsPresent):
                normativeFit(a, idxAgent)
                comparativeFit(a)
                salienceSocialGroup(a, idxAgent)
                salientActiveIdentity(a, idxAgent)
                listGraphics(a, s, r)
                playScenario(a, idxAgent) #added this function
                updateAccessibility(a) #update accessibility, no outcome evaluation was performed
            auxPlotScenario(s, r, True) #added this function
        clearSim()
        auxClearScenario() #added this function

In [ ]:
#Restarting the simulation
numSteps = 100
agents = prepareAgents(50, 0.5, True)
context = prepareContext(2)
addScenarioAgents(agents, True)
scenario = DictatorGame(20, 100)
cycleScenario3()
#Plots
print("Wealth Plots")
showAverageWealthAdaptingAgentsPlots()
showAverageWealthBoxPlotAdaptingAgents()